In [1]:
!sudo pip install -r requirements2.txt

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv('preprocessed.csv')

In [4]:
X = data.drop(['FINALDIAGNOSIS','DRINK',"SMOKE"], axis = 1)
y = data['FINALDIAGNOSIS']

In [5]:
X

,SEX,AGE,JOB,EDUCATION,MARRIAGE,CHARACTER1,CHARACTER2,CHARACTER3,CHARACTER4,CHARACTER5,...,BODYMEAS_8HAND6,BODYMEAS_8HAND7,BODYMEAS_8HAND8,SYSTOLICBP,DIASTOLICBP,GLUCOSE,T_CHOL,TG,HDL_CHOL,LDL_CHOL
0,2,48.666,2,5,2,2,2,1,2,1,...,82.0,91.0,96.0,120,80,104.0,219.0,72.0,60.0,128.0
1,2,80.773,14,3,2,1,1,1,1,1,...,92.0,96.0,99.0,124,65,110.0,199.0,97.0,46.0,114.0
2,1,50.008,3,3,2,3,1,1,1,1,...,80.0,81.0,81.0,121,75,79.0,216.0,88.0,49.0,132.0
3,2,50.551,14,4,2,1,1,1,1,1,...,91.0,94.0,99.0,113,81,100.0,218.0,136.0,44.0,136.0
4,1,49.581,8,2,2,3,2,3,2,2,...,98.0,98.0,100.0,129,72,92.0,186.0,61.0,33.0,118.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,50.682,3,5,2,3,1,2,2,1,...,74.5,80.0,88.0,120,80,71.0,190.0,75.0,73.0,101.0
496,2,42.542,3,5,2,3,1,3,3,1,...,64.5,76.0,79.0,110,70,74.0,246.0,86.0,104.0,117.0
497,1,48.797,7,5,2,3,2,2,2,2,...,95.0,96.5,97.5,150,100,125.0,246.0,220.0,74.0,154.0
498,1,51.263,7,4,2,3,2,1,1,1,...,86.0,89.5,91.4,110,80,85.0,172.0,127.0,49.0,109.0


In [6]:
from sklearn.preprocessing import StandardScaler
data_scale = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns, index = X.index)

In [7]:
# Splitting data into 80% training and 20% testing and validation (10% each)
X_train, X_temp, y_train, y_temp = train_test_split(data_scale, y, test_size=0.2, random_state=42, stratify = y)

# Convert the arrays to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_temp = torch.tensor(X_temp.values, dtype=torch.float32)
y_temp = torch.tensor(y_temp.values, dtype=torch.long)

# Splitting the remaining 20% into 50% testing and 50% validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify = y_temp)

In [8]:
class Model(nn.Module):
    def __init__(self, in_features = 69, h1 = 120, h2 = 60, h3 = 30, h4 = 15, h5 = 8, out_features = 4):
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, h4)
        self.fc5 = nn.Linear(h4, h5)
        self.out = nn.Linear(h5, out_features)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.out(x)

        return x

In [9]:
torch.manual_seed(32)
model = Model()

In [10]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epochs = 100000
train_losses = []
val_losses = []
best_val_loss = float('inf')  # 초기값으로 무한대 설정

for i in range(epochs):
    model.train()
    y_pred = model(X_train)

    loss = criterion(y_pred, y_train)
    train_losses.append(loss.item())

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        y_val_pred = model(X_val)
        val_loss = criterion(y_val_pred, y_val)
        val_losses.append(val_loss.item())

    if i % 1000 == 0:
        print(f'epoch {i}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

    # 모델의 성능이 더 좋아지면 모델 저장
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        torch.save(model.state_dict(), 'best_model.pth')  # 모델을 'best_model.pth' 파일로 저장

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch 0, Training Loss: 1.3309842348098755, Validation Loss: 1.3318294286727905


In [ ]:
X_test = StandardScaler().fit_transform(X_test)

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = y_test

In [ ]:
model = Model()
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

In [ ]:
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted_labels = torch.max(outputs, 1)

In [ ]:
correct_predictions = (predicted_labels == y_test_tensor).sum().item()
total_samples = y_test_tensor.size(0)
accuracy = (correct_predictions / total_samples) * 100

print(f'Model Accuracy on Test Data: {accuracy:.2f}%')